In [ ]:
import os
import numpy as np
import csv
import collections
from itertools import permutations

In [ ]:
filename = "/data/jazzy/Embeddings/datasets/CIKM/train-item-views.csv"
with open(filename) as f:
    reader = csv.reader(f, delimiter=";")
    sessions = collections.defaultdict(list)
    for i,r in enumerate(reader):
        if i == 0: continue;
        sessions[r[0]].append(r[2])
    print(sessions['1'])

In [ ]:
filetxt = "../data/productseq.txt"
transitive_file = "../data/items_transitive.tsv"
with open(filetxt, 'w') as f:
    for v in sessions.values():
        if len(v) > 1:
            for i in range(1,len(v)):
                [(v[i],v[j]) if for j in range(i)]
            f.write('%s\n' %(' '.join(v)))
            

In [ ]:
emb_file = "../output/product_emb.txt"
prod_categories = "/data/jazzy/Embeddings/datasets/CIKM/product-categories.csv"
products_file = "../log/metadata_prod.tsv"

embeddings = np.loadtxt(emb_file)
prod_dict = {}
rev_prod_dict = {}
with open(products_file) as p:
    reader = csv.reader(p)
    for i,pid in enumerate(reader):
        prod_dict[pid[0]] = i
        rev_prod_dict[i] = pid[0]
prod_cat = {}
with open(prod_categories) as cat:
    reader = csv.reader(cat, delimiter=";")
    for pid, cat in reader:
        prod_cat[pid] = cat
        

In [ ]:
cat = set(prod_cat.values())
print(len(cat))
print(len(prod_cat))

In [167]:
item_id = '13851'#36602
item_emb = embeddings[prod_dict[item_id]]
sim = np.matmul(embeddings, item_emb)
sim *= -1
top_k = 10  # number of nearest neighbors
nearest = np.array(sim).argsort()[1:top_k + 1]
print('Nearest to %s(%s):' % (item_id,prod_cat[item_id]))
print('Item_ID \t Category_ID')
for k in xrange(top_k):
    close_word = rev_prod_dict[nearest[k]]
    print('%s \t\t %s' % (close_word, prod_cat[close_word]))

Nearest to 13851(822):
Item_ID 	 Category_ID
258398 		 855
30395 		 139
88173 		 1180
80916 		 54
35334 		 278
172779 		 355
13803 		 822
36706 		 822
90306 		 822
1060 		 525


In [ ]:
from sklearn.neighbors import NearestNeighbors
neighbors = NearestNeighbors(n_neighbors=10, metric='euclidean', n_jobs=-1)
temp = neighbors.fit(embeddings)

In [166]:
item_id = '36602'
item_emb = embeddings[prod_dict[item_id]]
top_k = temp.kneighbors([item_emb], 11, return_distance=False)
print('Nearest to %s(%s):' % (item_id,prod_cat[item_id]))
print('Item_ID \t Category_ID')
for k in top_k[0,1:]:
    close_word = rev_prod_dict[k]
    print('%s \t\t %s' % (close_word, prod_cat[close_word]))


Nearest to 36602(962):
Item_ID 	 Category_ID
193720 		 474
3596 		 962
47373 		 962
68976 		 962
10858 		 962
206054 		 199
49576 		 581
324595 		 132
383444 		 633
102597 		 1287


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib import offsetbox
from mpl_toolkits.mplot3d import Axes3D

def plot_with_labels(low_dim_embs, labels, filename):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  fig = plt.figure(figsize=(18, 18))  # in inches
  ax = fig.add_subplot(111, projection='3d')

  num_cat = len(set(prod_cat.values()))
    
  for i, label in enumerate(labels):
    if label == 'UNK': continue
    x, y, z = low_dim_embs[i, :]
    ax.scatter(x, y, z, color=plt.cm.gist_ncar((int(prod_cat[label])) / (num_cat * 1.)))
    #ax.text(x, y, z,
        #prod_cat[label],#label,
        #xytext=(5, 2),
        #textcoords='offset points',
        #zorder = 1,)
        #color=plt.cm.gist_ncar((int(prod_cat[label])) / (num_cat * 1.)))
        #ha='right',
        #va='bottom',
        

  plt.savefig(filename)

def runTSNE(embeddings, reverse_dictionary, tsneimg_file):
    tsne = TSNE(
      perplexity=30, n_components=3, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join(os.getcwd(), tsneimg_file))
    
runTSNE(embeddings, rev_prod_dict, "tsne_prod.png")

In [ ]:
print(prod_cat['36411'])
print(prod_cat['7100'])
print(prod_cat['6080'])